In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('../../FinancesSP500.db')
c = conn.cursor()

In [3]:
brupt = pd.read_csv('Resources/BankruptcyData.csv')
brupt.drop_duplicates(keep='first',inplace=True)
name = 'BANKRUPTCY'
c.execute('drop table if exists '+ name)
brupt.to_sql(name,conn)

/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2788: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [4]:
dbase = pd.read_sql('select * from G1_CV_SYMBOLS',con=conn)

In [5]:
dbaseC = dbase[dbase.name.notnull()]
dbaseC.name = dbaseC.name.apply(lambda x: x.lower())

/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [6]:
companies = brupt['Debtor Name'].apply(lambda x: x.lower())

In [22]:
sorted(brupt['Industry Group'].apply(lambda x: x[:2]).value_counts().index)

['01',
 '10',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '20',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '41',
 '42',
 '44',
 '45',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '67',
 '70',
 '72',
 '73',
 '75',
 '76',
 '78',
 '79',
 '80',
 '82',
 '83',
 '87',
 '89']

In [200]:
pd.read_sql('''
select * 
from G1_CV_SYMBOLS
where name in (select '%' || "Debtor Name" || ''
                from BANKRUPTCY);
''',con=conn)

,symbol,name,price,exchange


In [182]:
pd.read_sql(
'''
select * 
from G1_CV_SYMBOLS
where name like '%{}%';
'''
.format('a')
,con=conn)

,symbol,name,price,exchange
0,CMCSA,Comcast Corp,52.72,Nasdaq Global Select
1,KMI,Kinder Morgan Inc,14.71,New York Stock Exchange
2,GDX,VanEck Vectors Gold Miners,31.13,NYSE Arca
3,GE,General Electric Co,12.56,New York Stock Exchange
4,BAC,Bank of America Corp,34.75,New York Stock Exchange
...,...,...,...,...
23666,SBEAU,SilverBox Engaged Merger Corp I,10.07,NasdaqGS
23667,PPGH,Poema Global Holdings Corp.,0.00,Nasdaq
23668,ASZ-UN,Austerlitz Acquisition Corporation II,10.25,NYSE
23669,EAP.CN,Euro Asia Pay Holdings Inc.,0.27,Canadian Sec


In [195]:
pd.read_sql('''
select "Debtor Name"
from BANKRUPTCY
''',con=conn)

,Debtor Name
0,1st Centennial Bancorp
1,"21st Century Oncology Holdings, Inc."
2,"A. H. Robins Company, Inc."
3,A. M. Castle & Co.
4,"A123 Systems, Inc."
...,...
1204,York Research Corporation (2002)
1205,"Young Broadcasting, Inc."
1206,Zale Corp.
1207,Zenith Electronics Corp.


In [125]:
%%time
i = 1
l = {}
for name1,symbol in zip(dbaseC.name,dbaseC.symbol):
    for name2 in companies:
        if SequenceMatcher(None, name1.strip(), name2.strip()).ratio() > .9:
            l.update({symbol:[name1.strip(),name2.strip()]})
            i += 1
            print(name1,name2)

halcon resources corporation halcon resources corporation
chesapeake energy corporation chesapeake energy corporation
delta air lines inc delta air lines, inc.
denbury resources inc denbury resources inc.
oasis petroleum inc oasis petroleum inc.
mcdermott international inc. mcdermott international, inc.
armstrong world industries inc armstrong world industries, inc.
magnachip semiconductor corp magnachip semiconductor llc
noble corporation plc noble corporation plc
newmark group inc newark group, inc.
mallinckrodt plc mallinckrodt plc
superior energy services inc superior energy services, inc.
sanchez energy corporation sanchez energy corporation
dean foods company dean foods company
charter communications inc charter communications, inc.
california resources corporation california resources corporation
extraction oil & gas inc. extraction oil & gas, inc.
diamond offshore drilling inc. diamond offshore drilling, inc.
akorn inc. akorn, inc.
pier 1 imports inc. pier 1 imports, inc.
owens

In [54]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [204]:
match = 0
nonmatch = 0
for k in l.keys():
    if l.get(k)[0] == l.get(k)[1]:
        #print('MATCHING:',l.get(k))
        match += 1
    else:
        print('NONMATCHING:',l.get(k))
        nonmatch += 1
print('Match total: ',match) ###
print('Nonmatch total: ',nonmatch) ###there has to be more

NONMATCHING: ['delta air lines inc', 'delta air lines, inc.']
NONMATCHING: ['denbury resources inc', 'denbury resources inc.']
NONMATCHING: ['oasis petroleum inc', 'oasis petroleum inc.']
NONMATCHING: ['mcdermott international inc.', 'mcdermott international, inc.']
NONMATCHING: ['armstrong world industries inc', 'armstrong world industries, inc.']
NONMATCHING: ['magnachip semiconductor corp', 'magnachip semiconductor llc']
NONMATCHING: ['newmark group inc', 'newark group, inc.']
NONMATCHING: ['superior energy services inc', 'superior energy services, inc.']
NONMATCHING: ['charter communications inc', 'charter communications, inc.']
NONMATCHING: ['extraction oil & gas inc.', 'extraction oil & gas, inc.']
NONMATCHING: ['diamond offshore drilling inc.', 'diamond offshore drilling, inc.']
NONMATCHING: ['akorn inc.', 'akorn, inc.']
NONMATCHING: ['pier 1 imports inc.', 'pier 1 imports, inc.']
NONMATCHING: ['ross stores inc', 'roses stores inc.']
NONMATCHING: ['abm industries inc', 'bmc indu

'cvs health corp'